# Invalid IPython Notebook Fixer with File Picker

This notebook fixes GitHub's "Invalid IPython Notebook" error using a simple file picker approach.

In [ ]:
# Install required packages if needed
# !pip install ipywidgets nbformat

In [ ]:
import json
import io
import base64
from datetime import datetime
from IPython.display import HTML, display
import ipywidgets as widgets

## Step 1: Define the Fix Function

In [ ]:
def fix_notebook_json(content):
    """Add the required widgets metadata to a notebook JSON"""
    try:
        # Parse the notebook
        notebook = json.loads(content)
        
        # Ensure metadata exists
        if 'metadata' not in notebook:
            notebook['metadata'] = {}
            
        # Add widgets metadata
        notebook['metadata']['widgets'] = {
            'application/vnd.jupyter.widget-state+json': {
                'state': {},
                'version_major': 2,
                'version_minor': 0
            }
        }
        
        # Convert back to JSON
        fixed_content = json.dumps(notebook, indent=2)
        return True, fixed_content
    except Exception as e:
        return False, f"Error fixing notebook: {str(e)}"

def create_download_link(content, filename):
    """Create an HTML download link for the fixed notebook"""
    b64 = base64.b64encode(content.encode()).decode()
    payload = f"data:application/json;base64,{b64}"
    html = f'''
    <div style="margin-top: 20px; padding: 10px; background-color: #f0f7ff; border-radius: 5px;">
        <h3 style="color: #0066cc;">✅ Notebook Fixed Successfully!</h3>
        <p>Click below to download the fixed notebook:</p>
        <a download="{filename}" 
           href="{payload}" 
           style="display: inline-block; background-color: #4CAF50; color: white; padding: 10px 20px; 
                  text-decoration: none; border-radius: 4px; font-weight: bold;">
            Download Fixed Notebook
        </a>
        <p style="margin-top: 10px;"><b>Note:</b> After downloading, replace your original notebook with this fixed version.</p>
    </div>
    '''
    return HTML(html)

## Step 2: Create the File Input Element

The code below creates an HTML file input that works in any Jupyter environment.

In [ ]:
file_upload_html = '''
<div style="margin: 20px 0; padding: 15px; background-color: #f5f5f5; border-radius: 5px;">
    <h3 style="margin-top: 0;">Select Your Notebook File</h3>
    <input type="file" id="notebook_file" accept=".ipynb" 
           style="margin: 10px 0; padding: 10px; border: 1px solid #ccc; border-radius: 4px; width: 100%;">
    <div id="file_info" style="margin: 10px 0; color: #555;"></div>
    <button id="process_button" disabled
            style="background-color: #4CAF50; color: white; padding: 10px 20px; 
                   border: none; border-radius: 4px; cursor: pointer; font-weight: bold;">
        Fix Notebook
    </button>
    <div id="status" style="margin-top: 15px;"></div>
    <div id="download_area"></div>
</div>

<script>
    // Get elements
    const fileInput = document.getElementById('notebook_file');
    const fileInfo = document.getElementById('file_info');
    const processButton = document.getElementById('process_button');
    const statusDiv = document.getElementById('status');
    const downloadArea = document.getElementById('download_area');
    
    // File selection handler
    fileInput.addEventListener('change', function(e) {
        const file = fileInput.files[0];
        if (file) {
            fileInfo.innerHTML = `Selected file: <strong>${file.name}</strong> (${(file.size / 1024).toFixed(2)} KB)`;
            processButton.disabled = false;
        } else {
            fileInfo.innerHTML = '';
            processButton.disabled = true;
        }
    });
    
    // Process button handler
    processButton.addEventListener('click', function() {
        const file = fileInput.files[0];
        if (!file) return;
        
        statusDiv.innerHTML = '<p style="color: #0066cc;">Reading notebook file...</p>';
        
        const reader = new FileReader();
        reader.onload = function(e) {
            try {
                statusDiv.innerHTML = '<p style="color: #0066cc;">Processing notebook...</p>';
                const notebookContent = e.target.result;
                
                // Parse notebook and add required metadata
                const notebook = JSON.parse(notebookContent);
                
                // Ensure metadata exists
                if (!notebook.metadata) {
                    notebook.metadata = {};
                }
                
                // Add widgets metadata
                notebook.metadata.widgets = {
                    'application/vnd.jupyter.widget-state+json': {
                        'state': {},
                        'version_major': 2,
                        'version_minor': 0
                    }
                };
                
                // Create fixed content
                const fixedContent = JSON.stringify(notebook, null, 2);
                
                // Create timestamp for filename
                const timestamp = new Date().toISOString().replace(/[:.]/g, '-').slice(0, 19);
                const originalName = file.name;
                const fixedName = `fixed_${timestamp}_${originalName}`;
                
                // Create download link
                const blob = new Blob([fixedContent], {type: 'application/json'});
                const url = URL.createObjectURL(blob);
                
                // Success message and download link
                statusDiv.innerHTML = '<p style="color: green; font-weight: bold;">✅ Notebook fixed successfully!</p>';
                downloadArea.innerHTML = `
                    <div style="margin-top: 15px;">
                        <a href="${url}" download="${fixedName}" 
                           style="display: inline-block; background-color: #4CAF50; color: white; 
                                  padding: 10px 20px; text-decoration: none; border-radius: 4px; 
                                  font-weight: bold;">
                            Download Fixed Notebook
                        </a>
                        <p style="margin-top: 10px;"><b>Note:</b> Save this file and replace your original notebook.</p>
                    </div>
                `;
                
            } catch (error) {
                statusDiv.innerHTML = `<p style="color: red;">❌ Error: ${error.message}</p>`;
                downloadArea.innerHTML = '';
            }
        };
        
        reader.onerror = function() {
            statusDiv.innerHTML = '<p style="color: red;">❌ Error reading file</p>';
            downloadArea.innerHTML = '';
        };
        
        reader.readAsText(file);
    });
</script>
'''

display(HTML(file_upload_html))

## How It Works

1. Click the **Select Your Notebook File** button and choose your `.ipynb` file
2. Click the **Fix Notebook** button
3. The notebook will be processed in your browser (no server-side processing)
4. A download link will appear for the fixed version
5. Download the fixed notebook and replace your original file

### What Gets Fixed

This tool adds the following metadata structure to your notebook that GitHub requires:

```json
"metadata": {
  "widgets": {
    "application/vnd.jupyter.widget-state+json": {
      "state": {},
      "version_major": 2,
      "version_minor": 0
    }
  }
}
```

This resolves the common "Invalid IPython Notebook" error in GitHub's browser viewer.